In [3]:
#ffmpeg installed?
#install librosa on anaconda
import librosa                    
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg
import os
import matplotlib.pyplot as plt
import pandas as pd
import cv2

#### Generador de imagenes

In [4]:
def crop_borders(img,x1=35,x2=252,y1=54,y2=389):
    '''Corta los bordes blancos creados por matplotlib'''
    cropped = img[x1:x2,y1:y2]
    return cropped

In [5]:
DIR = './test_songs/'

def save_img(file):
    image_name = './' + (file.split('/')[2][:-4]) + "_BnW"
    print(image_name)
    n_mels = 128
    hop_length = 512
    n_fft = 2048

    file_location = file
    y, sr = librosa.load(file_location, duration=30.0)
    song,_ = librosa.effects.trim(y)

    S = librosa.feature.melspectrogram(song, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
    S_DB = librosa.power_to_db(S, ref=np.max)
    S_DB[S_DB < -40] = -40
    plt.figure(figsize=(432/80, 288/80), dpi=80)
    disp = librosa.display.specshow(S_DB, sr=sr, hop_length=hop_length, x_axis='time', y_axis='mel');
    plt.axis('off')
    plt.savefig('./test_songs/images/'+image_name)
    plt.close()

    finale_name = './test_songs/images/'+image_name+'.png'
    img = cv2.imread(finale_name,0)
    cv2.imwrite(finale_name, crop_borders(img))
    
files = os.listdir('./test_songs')
songs = [ img for img in files if img.split('.')[-1] == 'mp3']
print(songs)

for song in songs:
    print(DIR + song)
    save_img(DIR + song)

['abba.mp3', 'desakato.mp3', 'elfinde.mp3', 'harder.mp3', 'iseo.mp3', 'marley.mp3', 'metallica.mp3', 'paradise.mp3', 'requiem.mp3', 'sultans.mp3', 'wutang.mp3']
./test_songs/abba.mp3
./abba_BnW


C:\Users\Alberto\Anaconda3\lib\site-packages\librosa\core\audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


./test_songs/desakato.mp3
./desakato_BnW
./test_songs/elfinde.mp3
./elfinde_BnW
./test_songs/harder.mp3
./harder_BnW
./test_songs/iseo.mp3
./iseo_BnW
./test_songs/marley.mp3
./marley_BnW
./test_songs/metallica.mp3
./metallica_BnW
./test_songs/paradise.mp3
./paradise_BnW
./test_songs/requiem.mp3
./requiem_BnW
./test_songs/sultans.mp3
./sultans_BnW
./test_songs/wutang.mp3
./wutang_BnW


In [6]:
def save_mel(file, directory):
    '''Guarda la imagen del espectograma de mel'''
    n_mels = 128
    hop_length = 512
    n_fft = 2048

    file_location = file
    y, sr = librosa.load(file_location, duration=30.0)
    song,_ = librosa.effects.trim(y)

    S = librosa.feature.melspectrogram(song, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
    S_DB = librosa.power_to_db(S, ref=np.max)
    S_DB[S_DB < -40] = -40
    plt.figure(figsize=(432/80, 288/80), dpi=80)
    disp = librosa.display.specshow(S_DB, sr=sr, hop_length=hop_length, x_axis='time', y_axis='mel');
    plt.axis('off')
    plt.savefig(directory)
    plt.close()

In [7]:
def cut_n_save(file, flag = 1):
    '''Guarda y vuelve a guardar en su ubicacion'''
    img = cv2.imread(file,flag)
    cv2.imwrite(file, crop_borders(img))

In [8]:
def ensure_dir(file_path):
    '''Asegura que exista el path, si no lo crea'''
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

In [10]:
IMG = './data/images_original'
MUSIC = './data/genres_original'
DIR = './dataset/'

for root, dirs, files in os.walk(MUSIC):
    for name in files:
        filename = os.path.join(root, name)
        if(filename != "./data/genres_original\jazz\jazz.00054.wav"): #corrupted
            genre = filename.split('\\')[-2]
            fn = filename.split('\\')[-1].split('.')

            target = fn[0] + fn[1] + '.png'

            output = DIR + genre + '/' + target

            ensure_dir(output)
            save_mel(filename, output)
            cut_n_save(output)

#### Generador de más características de las canciones

In [10]:
#Funciones para obtener las caracteristicas de las canciones del dataset
#Obtenemos 133 caracteristicas, mas de el doble que de las que teniamos antes

def get_features(y, sr, n_fft = 1024, hop_length = 512):
    # Features to concatenate in the final dictionary
    features = {'centroid': None, 'roloff': None, 'flux': None, 'rmse': None,
                'zcr': None, 'contrast': None, 'bandwidth': None, 'flatness': None}
    
    # Count silence
    if 0 < len(y):
        y_sound, _ = librosa.effects.trim(y, frame_length=n_fft, hop_length=hop_length)
    features['sample_silence'] = len(y) - len(y_sound)

    # Using librosa to calculate the features
    features['centroid'] = librosa.feature.spectral_centroid(y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    features['roloff'] = librosa.feature.spectral_rolloff(y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    features['zcr'] = librosa.feature.zero_crossing_rate(y, frame_length=n_fft, hop_length=hop_length).ravel()
    features['rmse'] = librosa.feature.rms(y, frame_length=n_fft, hop_length=hop_length).ravel()
    features['flux'] = librosa.onset.onset_strength(y=y, sr=sr).ravel()
    features['contrast'] = librosa.feature.spectral_contrast(y, sr=sr).ravel()
    features['bandwidth'] = librosa.feature.spectral_bandwidth(y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    features['flatness'] = librosa.feature.spectral_flatness(y, n_fft=n_fft, hop_length=hop_length).ravel()
    
    # MFCC treatment
    mfcc = librosa.feature.mfcc(y, n_fft = n_fft, hop_length = hop_length, n_mfcc=13)
    for idx, v_mfcc in enumerate(mfcc):
        features['mfcc_{}'.format(idx)] = v_mfcc.ravel()
        
    # Get statistics from the vectors
    def get_moments(descriptors):
        result = {}
        for k, v in descriptors.items():
            result['{}_max'.format(k)] = np.max(v)
            result['{}_min'.format(k)] = np.min(v)
            result['{}_mean'.format(k)] = np.mean(v)
            result['{}_std'.format(k)] = np.std(v)
            result['{}_kurtosis'.format(k)] = kurtosis(v)
            result['{}_skew'.format(k)] = skew(v)
        return result
    
    dict_agg_features = get_moments(features)
    dict_agg_features['tempo'] = librosa.beat.tempo(y, sr=sr)[0]
    
    return dict_agg_features

def read_process_songs(src_dir, debug = True):    
    # Empty array of dicts with the processed features from all files
    arr_features = []

    # Read files from the folders
    for x,_ in genres.items():
        folder = src_dir + x
        
        for root, subdirs, files in os.walk(folder):
            for file in files:
                # Read the audio file
                print(file)
                if(file != "jazz.00054.wav"): #archivo corrupto
                    file_name = folder + "/" + file
                    signal, sr = librosa.load(file_name)

                    # Debug process
                    if debug:
                        print("Reading file: {}".format(file_name))

                    # Append the result to the data structure
                    features = get_features(signal, sr)
                    features['genre'] = genres[x]
                    arr_features.append(features)
    return arr_features

In [10]:
#Devuelve un dataframe con las características extendidas de las canciones
#Debido a que tarda en ejecutar ~10 min cada vez, hemos exportado a un csv las caracteristicas extra para acceder más rapido
# %%time
# # Get list of dicts with features and convert to dataframe
# gtzan_dir = './data/genres_original/'
# features = read_process_songs(gtzan_dir, debug=False) #10min aprox to execute
# df_features = pd.DataFrame(features)